In [ ]:
# Usuf Submission- Internship submission

In [ ]:
# Uploading the slected data from opus amd extraction. I have selected english french language pair

# !unzip en-fr.txt.zip
# !wc -l Op*
# 216418--raw data

In [1]:
#  cleaning the data

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

def write_file(file_path, lines):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def remove_duplicates(source_lines, target_lines):
    combined = list(set(zip(source_lines, target_lines)))
    unique_source, unique_target = zip(*combined)
    return list(unique_source), list(unique_target)

def filter_by_length(source_lines, target_lines, max_words=200):
    filtered_source, filtered_target = [], []
    for source, target in zip(source_lines, target_lines):
        if len(source.split()) <= max_words and len(target.split()) <= max_words:
            filtered_source.append(source)
            filtered_target.append(target)
    return filtered_source, filtered_target

def filter_by_ratio(source_lines, target_lines, ratio_threshold=1.5):
    filtered_source, filtered_target = [], []
    for source, target in zip(source_lines, target_lines):
        source_len = len(source.split())
        target_len = len(target.split())
        if source_len > 0 and target_len > 0:
            if (source_len / target_len <= ratio_threshold) and (target_len / source_len <= ratio_threshold):
                filtered_source.append(source)
                filtered_target.append(target)
    return filtered_source, filtered_target

def clean_files(source_file, target_file, output_source_file, output_target_file):
    source_lines = read_file(source_file)
    target_lines = read_file(target_file)

    assert len(source_lines) == len(target_lines), "Source and target files must have the same number of lines."

    source_lines, target_lines = remove_duplicates(source_lines, target_lines)
    source_lines, target_lines = filter_by_length(source_lines, target_lines)
    source_lines, target_lines = filter_by_ratio(source_lines, target_lines)

    write_file(output_source_file, source_lines)
    write_file(output_target_file, target_lines)

clean_files('OpenSubtitles.en-fr.en', 'OpenSubtitles.en-fr.fr', 'cleaned_text.en', 'cleaned_text.fr')




In [2]:
# After cleaning-141711
# !wc -l clean*


In [4]:
#  splitting the test data and the train+development (we split later) data

import random

def split_train_test(source_lines, target_lines, train_ratio=0.98):
    assert len(source_lines) == len(target_lines), "Source and target lists must have the same number of lines."
    combined = list(zip(source_lines, target_lines))
    random.shuffle(combined)

    train_size = int(len(combined) * train_ratio)

    train_data = combined[:train_size]
    test_data = combined[train_size:]

    train_source, train_target = zip(*train_data)
    test_source, test_target = zip(*test_data)

    return list(train_source), list(train_target), list(test_source), list(test_target)


In [5]:
# we have 2835 test sentences
source_lines = read_file('cleaned_text.en')
target_lines = read_file('cleaned_text.fr')
train_src, train_trg, test_src, test_trg = split_train_test(source_lines, target_lines)


In [6]:
len(test_trg)

2835

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_top_50k_sentences(train_src, train_trg, test_src):
    vectorizer = CountVectorizer().fit(train_src + test_src)

    train_vectors = vectorizer.transform(train_src)
    test_vectors = vectorizer.transform(test_src)

    # Compute cosine similarity between each training sentence and all test sentences
    similarities = cosine_similarity(train_vectors, test_vectors)

    max_similarities = np.max(similarities, axis=1)

    # Get the indices of the top 50,000 most similar training sentences
    top_50k_indices = np.argsort(max_similarities)[-50000:]

    top_train_src = [train_src[i] for i in top_50k_indices]
    top_train_trg = [train_trg[i] for i in top_50k_indices]

    return top_train_src, top_train_trg



top_train_src, top_train_trg = get_top_50k_sentences(train_src, train_trg, test_src)


In [8]:
len(top_train_src)

50000

In [9]:
!pip install datasets evaluate transformers[sentencepiece]==4.33.0

!pip install accelerate -U

!pip install faiss-cpu sentence-transformers
!pip install sacrebleu

  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
Using cached tokenizers-0.20.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [11]:
# loading Nllb model
Source_list=top_train_src
Target_list=top_train_trg

In [12]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

# Initialize the dictionary with structure for 'id' and 'translation'
my_dict = {'id': [], 'translation': []}
count = 0

for key, value in zip(Source_list, Target_list):
    # Assuming key and value do not require stripping newlines
    # Add the count and corresponding translation to the dictionary
    my_dict['id'].append(count)
    my_dict['translation'].append({'en': key, 'fr': value})
    count += 1

# Create the dataset from the dictionary
dataset = Dataset.from_dict(my_dict)

# Split the dataset into training and validation sets
train_dataset, validation_dataset = dataset.train_test_split(test_size=0.1).values()

# Combine the training and validation sets into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})


In [13]:
max_input_length = 150
max_target_length = 150
source_lang = "en"
target_lang = "fr"
prefix=""

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [35]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_checkpoint = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [15]:
# tokenizer

In [36]:
tokenizer.src_lang = "eng_Latn"  # English
tokenizer.tgt_lang = "fra_Latn"  # French
# tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

In [18]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    gradient_accumulation_steps=4    
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [21]:
from accelerate import Accelerator
from accelerate import DataLoaderConfiguration

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,0.463900,0.251366,31.018300,11.632000
1,0.241700,0.244833,31.128800,11.533800
2,0.227800,0.243366,31.521400,11.631200
4,0.213600,0.243547,31.454200,11.642800


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**c

TrainOutput(global_step=7030, training_loss=0.5549357303006212, metrics={'train_runtime': 7389.091, 'train_samples_per_second': 30.45, 'train_steps_per_second': 0.951, 'total_flos': 2.7734469253791744e+16, 'train_loss': 0.5549357303006212, 'epoch': 4.999111111111111})

In [37]:
trainer.state.best_model_checkpoint

'nllb-200-distilled-600M-finetuned-en-to-fr/checkpoint-5625'

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Specify the checkpoint directory or file path
checkpoint_path = "nllb-200-distilled-600M-finetuned-en-to-fr/checkpoint-5625//"

# Load the tokenizer from the checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Load the model from the checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to('cuda')

In [38]:
store=[]
for text in test_src:
    # print(text)
    tokenized_text = tokenizer(text, return_tensors="pt", padding=True).to('cuda')
    translated_tokens = model.generate(**tokenized_text,forced_bos_token_id=tokenizer.convert_tokens_to_ids("fra_Latn"))
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    # print(translated_text)
    # count=count+1
    store.append(translated_text)
    

In [39]:
!pwd

/home/ec2-user/SageMaker/Untitled Folder


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
with open("base_outout","w") as ref_file:
    for line in test_src:
        line = str(line).strip()  
        ref_file.write(line + "\n")

In [31]:
!wc -l Usuf_outout

2835 Usuf_outout


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
import sacrebleu

def calculate_bleu_and_ter(hypotheses, references):
    
    references = [[ref] for ref in references]

    bleu = sacrebleu.corpus_bleu(hypotheses, references)
    print(f"BLEU score: {bleu.score}")

    ter = sacrebleu.corpus_ter(hypotheses, references)
    print(f"TER score: {ter.score}")

hypotheses = store
references = test_trg
calculate_bleu_and_ter(hypotheses, references)


BLEU score: 35.35533905932737
TER score: 16.303410201851744


In [34]:
!du -h nllb-200-distilled-600M-finetuned-en-to-fr/

7.0G	nllb-200-distilled-600M-finetuned-en-to-fr/checkpoint-5625
7.0G	nllb-200-distilled-600M-finetuned-en-to-fr/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
